# Yelp Reviews

### Load data

In [1]:
import json
def _load_data(filename):
    with open(filename) as f:
        reviews = f.read().strip().split('\n')
    reviews = [json.loads(review) for review in reviews]
    return reviews

def _get_texts(reviews):
    texts = [review['text'] for review in reviews]
    return texts

def get_data_and_labels(filepath):
    reviews = _load_data(filepath)
    texts = _get_texts(reviews)

    binstars = [0 if review['stars'] <= 3 else 1 for review in reviews]
    balanced_texts = []
    balanced_labels = []
    limit = 100000  # Change this to grow/shrink the dataset
    neg_pos_counts = [0, 0]
    for i in range(len(texts)):
        polarity = binstars[i]
        if neg_pos_counts[polarity] < limit:
            balanced_texts.append(texts[i])
            balanced_labels.append(binstars[i])
            neg_pos_counts[polarity] += 1

    return balanced_texts, balanced_labels, None

In [2]:
texts, labels, _ = get_data_and_labels('../data/review_100_samples.json')

In [3]:
from sklearn.model_selection import train_test_split

data_train, data_val, labels_train, labels_val = train_test_split(texts, labels, random_state=0)

### Train model

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', RandomForestClassifier())
                    ])
text_clf = text_clf.fit(data_train, labels_train)

### Evaluate model

In [5]:
from sklearn.metrics import precision_score, recall_score
y_pred = text_clf.predict(data_val)
y_true = labels_val
p = precision_score(y_true, y_pred)
r = recall_score(y_true, y_pred)
print('precision score: {}'.format(p))
print('recall score: {}'.format(r))

precision score: 0.6086956521739131
recall score: 1.0


In [6]:
text_clf.predict(["No no no no no "])

array([1])

In [65]:
# import pickle
# pickle.dump(text_clf, open('./model.pkl', 'wb'))